In [1]:
# this introduces Salesforce's CodeGen model which is a pretrained 
# model for generating code

In [3]:
import torch
import torch.nn as nn
import numpy as np

device =  torch.device("mps" if torch.backends.mps.is_available() else "cpu")

Import and inspect the CodeGen model

In [6]:
# Model source
# https://huggingface.co/Salesforce/codegen-350M-mono   [this model is only trained on Python - mono]

from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained('Salesforce/codegen-350M-mono')

model = AutoModelForCausalLM.from_pretrained('Salesforce/codegen-350M-mono').to(device)

/Users/raeez/.pyenv/versions/jupyter-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading weights: 100%|█| 165/165 [00:00<00:00, 1876.00it/s, Materializing param=
CodeGenForCausalLM LOAD REPORT from: Salesforce/codegen-350M-mono
Key                                     | Status     |  | 
----------------------------------------+------------+--+-
transformer.h.{0...19}.attn.causal_mask | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [10]:
tokenizer.vocab_size

50257

In [11]:
model
# the model has 50k tokens, and embedd dimension in 1024
# it doesnt have position embeddings
# it has  20 transformer blocks


CodeGenForCausalLM(
  (transformer): CodeGenModel(
    (wte): Embedding(51200, 1024)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-19): 20 x CodeGenBlock(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): CodeGenAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (qkv_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=False)
        )
        (mlp): CodeGenMLP(
          (fc_in): Linear(in_features=1024, out_features=4096, bias=True)
          (fc_out): Linear(in_features=4096, out_features=1024, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=51200, bias=True)
)

Check out the tokenizer

In [15]:
print(f'Tokenizer has {tokenizer.vocab_size} tokens')

for i in range(30):
    randtok = torch.randint(tokenizer.vocab_size,(1,))
    # print(randto)
    print(f'Token {randtok[0]:5} is "{tokenizer.decode(randtok)}"')

Tokenizer has 50257 tokens
Token 21223 is "affe"
Token  5724 is " rot"
Token  6110 is " defensive"
Token 36797 is " haste"
Token  6248 is "sen"
Token  5545 is " presented"
Token 21230 is "safe"
Token  9617 is " threw"
Token 18861 is "Bob"
Token  7010 is " isol"
Token 39929 is " Reese"
Token 37382 is "addle"
Token  4797 is "iliar"
Token  6828 is "ounter"
Token 23691 is " Push"
Token 35794 is " Latvia"
Token 42605 is " Rita"
Token 42791 is " Refugee"
Token 49166 is " analytic"
Token  3402 is " shown"
Token  5980 is " dial"
Token 38559 is " LIC"
Token 48494 is "Whereas"
Token 24590 is " laps"
Token 22245 is "Richard"
Token  9079 is " requirement"
Token 45702 is " grinning"
Token 44767 is " bulldo"
Token 36158 is " Augustine"
Token 10365 is "opp"


In [16]:
randtok

tensor([10365])

In [17]:
# The tokenizer has 50257 tokens in its vocab
# But the model's embedding matrix is of size 51200x1024 (not 50257)
# embeding matrix larger than the vocab size of tokenizer
     # that means there are several rows in the embedd matrix that have no connection to the data
     #they get randomly modified during backprop, but dont correspond to any tokens
     # maybe to fit the matrix in GPU

Genereate a response to some code input

In [20]:
text = 'for i in range(10):'
input_ids = tokenizer(text, return_tensors='pt').input_ids.to(device)

generated_ids = model.generate(input_ids, max_length=222,temperature=1.4, do_sample=True)
print(tokenizer.decode(generated_ids[0],skip_special_tokens=True))
                               

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


for i in range(10):
            bounds = self.Bounds.allocateOne(self.n_agents)
            Bounds.constrains(i)[0]).attach(self.aloc_d.location_vector() == (randn()) | bounds.location.getRotationalFrame(),i).draw(*bounds,2,2,'+',(False,True.5),self.placements.clear())##,color=[0.1,0.2,0.3]+1e9)
            # for loc in ['r', None, 10.4*i] do B.n_aloc_dim(s[loc+'s'].getNumActions) if true
        Bounds.boundsFrom('r', (False,True))
        Bounds(s1.agent, loc=10).setOrientalMatrix((False,True))
        Boms = [BoxMotionSaver('rzw',
                                 n_domains=s


Now import git-calc files from github so that we can finetune the model on that codebas

In [ ]:
import os
import subprocess
import nbformat

# download the repo
repo_url = 'htpps://github.com/mikexcohen/Calculus_book.git'
repo_dir = 'Calci